In [ ]:
!pip install transformers sentencepiece torch


In [ ]:
from transformers import WhisperProcessor, WhisperForConditionalGeneration
import torch
import torchaudio

# Load model and processor
model_name = "openai/whisper-medium"
processor = WhisperProcessor.from_pretrained(model_name)
model = WhisperForConditionalGeneration.from_pretrained(model_name)

In [ ]:
# Load audio
speech_array, sr = torchaudio.load("/content/TunePocket-60-Sec-Count-Up-Counter-Female-48kHz-Preview.mp3")
speech = speech_array.mean(dim=0)  # convert to mono

# Resample to 16 kHz (required by Whisper)
if sr != 16000:
    resampler = torchaudio.transforms.Resample(orig_freq=sr, new_freq=16000)
    speech = resampler(speech)
    sr = 16000

/usr/local/lib/python3.12/dist-packages/torchaudio/_backend/utils.py:213: UserWarning: In 2.9, this function's implementation will be changed to use torchaudio.load_with_torchcodec` under the hood. Some parameters like ``normalize``, ``format``, ``buffer_size``, and ``backend`` will be ignored. We recommend that you port your code to rely directly on TorchCodec's decoder instead: https://docs.pytorch.org/torchcodec/stable/generated/torchcodec.decoders.AudioDecoder.html#torchcodec.decoders.AudioDecoder.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchaudio/_backend/ffmpeg.py:88: UserWarning: torio.io._streaming_media_decoder.StreamingMediaDecoder has been deprecated. This deprecation is part of a large refactoring effort to transition TorchAudio into a maintenance phase. The decoding and encoding capabilities of PyTorch for both audio and video are being consolidated into TorchCodec. Please see https://github.com/pytorch/audio/issues/3902 for more information. It will be r

In [ ]:
from transformers import WhisperProcessor, WhisperForConditionalGeneration
import torch
import torchaudio

# --- CONFIG ---
audio_file = "/content/TunePocket-60-Sec-Count-Up-Counter-Female-48kHz-Preview.mp3"
model_name = "openai/whisper-small"  # Use 'large' for higher accuracy if CPU/RAM allows
chunk_seconds = 10  # Chunk length in seconds
overlap_seconds = 2  # Overlap between chunks in seconds
output_txt_file = "transcription.txt"

# --- LOAD MODEL ---
processor = WhisperProcessor.from_pretrained(model_name)
model = WhisperForConditionalGeneration.from_pretrained(model_name)

# --- LOAD AUDIO ---
speech_array, sr = torchaudio.load(audio_file)
speech = speech_array.mean(dim=0)  # convert to mono

# Resample if needed
if sr != 16000:
    resampler = torchaudio.transforms.Resample(orig_freq=sr, new_freq=16000)
    speech = resampler(speech)
    sr = 16000

# --- SPLIT AUDIO INTO CHUNKS WITH OVERLAP ---
chunk_size = chunk_seconds * sr
overlap_size = overlap_seconds * sr

chunks = []
for start in range(0, speech.shape[0], chunk_size - overlap_size):
    chunk = speech[start:start + chunk_size]
    chunks.append(chunk)

# --- TRANSCRIBE EACH CHUNK ---
full_text = ""
for i, chunk in enumerate(chunks):
    inputs = processor(chunk, sampling_rate=sr, return_tensors="pt")
    generated_ids = model.generate(**inputs, task="translate", language="en")
    transcription = processor.batch_decode(generated_ids, skip_special_tokens=True)
    full_text += transcription[0].strip() + " "

# --- SAVE TO FILE ---
with open(output_txt_file, "w", encoding="utf-8") as f:
    f.write(full_text.strip())

print("✅ Full transcription saved to:", output_txt_file)
print("\n--- Transcription Preview ---\n")
print(full_text.strip())


preprocessor_config.json: 0.00B [00:00, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

...

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

generation_config.json: 0.00B [00:00, ?B/s]

✅ Full transcription saved to: transcription.txt

--- Transcription Preview ---

1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42. 41, 42, 43, 44, 45, 46, 47, 48, 49, 50. 49, 50, 51, 52, 53, 54, 55, 56, 57, 58. 57, 58, 59, 60.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os

drive_folder = "/content/drive/MyDrive/WhisperModels"
os.makedirs(drive_folder, exist_ok=True)


In [ ]:
from transformers import WhisperProcessor, WhisperForConditionalGeneration

model_name = "openai/whisper-small"

# Load model & processor
processor = WhisperProcessor.from_pretrained(model_name)
model = WhisperForConditionalGeneration.from_pretrained(model_name)

# Save model & processor to Drive
model.save_pretrained(drive_folder)
processor.save_pretrained(drive_folder)

print("✅ Model and processor saved to Google Drive:", drive_folder)

✅ Model and processor saved to Google Drive: /content/drive/MyDrive/WhisperModels


In [ ]:
from transformers import WhisperProcessor, WhisperForConditionalGeneration

drive_folder = "/content/drive/MyDrive/WhisperModels"

# Load model & processor from Drive
processor = WhisperProcessor.from_pretrained(drive_folder)
model = WhisperForConditionalGeneration.from_pretrained(drive_folder)

print("✅ Model loaded from Google Drive!")

OSError: Can't load feature extractor for '/content/drive/MyDrive/WhisperModels'. If you were trying to load it from 'https://huggingface.co/models', make sure you don't have a local directory with the same name. Otherwise, make sure '/content/drive/MyDrive/WhisperModels' is the correct path to a directory containing a preprocessor_config.json file